In [615]:
import gzip
import json
from datetime import datetime
import pandas as pd
import pandasql as ps
import numpy as np
from IPython.display import IFrame, display

#  **First: Review Existing Unstructured Data and Diagram a New Structured Relational Data Model**

Reviewing three JSON data files, I see we have receipts file for transactions data, brands file that store product/brand elements, and user file for customer data. I would start from **exploring and cleaning data** as it most of the time help me understand more about the data.

For user data, a lots of time I use it for segmentation. In other words, I would use it heavily at where statement in SQL query so I would like to keep the data straight forward. Otherwise, it would be hard to put it at **where** statement and also hard to read. Therefore, I convert role field to a boolean type field real_customer just to keep it clean as I only observe two type of value "staff user" and "customer". All the other field are pretty clean except for date are in milisecond, and I convert it to datetime format so it's clean and at the same time not losing any info.

In [616]:
# import json file to dataframe
users = []

with gzip.open('data/users.json.gz','rb') as r:
    for line in r:
        d = json.loads(line)
        users.append(d)
        
users = pd.DataFrame(users)

In [617]:
# convert role to boolean type data real_customer
users['real_customer'] = [role == "consumer" for role in users['role']]

users=users.drop(['role'],axis = 1)

In [618]:
# flatten nested distionary
users['_id'] = [d['$oid'] for d in users['_id']]

# change datetime format from milisecond to datetime
users['createdDate'] = [datetime.fromtimestamp(d['$date']/1000).strftime("%m-%d-%Y %I:%M:%S") for d in users['createdDate']]

users['lastLogin'] = [datetime.fromtimestamp(d['$date']/1000).strftime("%m-%d-%Y %I:%M:%S") if '$date' in d else 'no data' for d in users['lastLogin'].fillna('no data')]

The users data now looks like this

In [619]:
users

,_id,active,createdDate,lastLogin,signUpSource,state,real_customer
0,5ff1e194b6a9d73a3a9f1052,True,01-03-2021 07:24:04,01-03-2021 07:25:37,Email,WI,True
1,5ff1e194b6a9d73a3a9f1052,True,01-03-2021 07:24:04,01-03-2021 07:25:37,Email,WI,True
2,5ff1e194b6a9d73a3a9f1052,True,01-03-2021 07:24:04,01-03-2021 07:25:37,Email,WI,True
3,5ff1e1eacfcf6c399c274ae6,True,01-03-2021 07:25:30,01-03-2021 07:25:30,Email,WI,True
4,5ff1e194b6a9d73a3a9f1052,True,01-03-2021 07:24:04,01-03-2021 07:25:37,Email,WI,True
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,12-19-2014 06:21:22,03-05-2021 08:52:23,NaN,NaN,False
491,54943462e4b07e684157a532,True,12-19-2014 06:21:22,03-05-2021 08:52:23,NaN,NaN,False
492,54943462e4b07e684157a532,True,12-19-2014 06:21:22,03-05-2021 08:52:23,NaN,NaN,False
493,54943462e4b07e684157a532,True,12-19-2014 06:21:22,03-05-2021 08:52:23,NaN,NaN,False


For **brand data**, brand key and barcode both look like joinable key but some rows doesn't have brand code and every rows has barcode. Therefore, I decide to use barcode as joinable key, though both barcode and brandcode seems to have duplicate record. The data are pretty clean I just need to flatten dictionary value.

In [620]:
# import json file to dataframe
brands = []

with gzip.open('data/brands.json.gz','rb') as r:
    for line in r:
        d = json.loads(line)
        brands.append(d)
        
brands = pd.DataFrame(brands)

In [621]:
# flatten id and cpg id
brands['_id'] = [b['$oid'] for b in brands['_id']]
brands['cpg'] = [b['$id']['$oid'] for b in brands['cpg']]
brands = brands.rename({"cpg":"cpg_id","brandCode":"brand_code"}, axis=1)

In [622]:
# every rows has barcode
bar_code_test = brands.barcode.isnull().values.any()
print("is there any null barcode:",bar_code_test)

is there any null barcode: False


In [623]:
# not every rows has brandcode
brand_code_test = brands.brand_code.isnull().values.any()
print("is there any null brand_code:",brand_code_test)

is there any null brand_code: True


The brand data now looks like this

In [624]:
brands

,_id,barcode,category,categoryCode,cpg_id,name,topBrand,brand_code
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,601ac114be37ce2ead437550,test brand @1612366101024,False,NaN
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,Starbucks,False,STARBUCKS
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,601ac142be37ce2ead437559,test brand @1612366146176,False,TEST BRANDCODE @1612366146176
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,601ac142be37ce2ead437559,test brand @1612366146051,False,TEST BRANDCODE @1612366146051
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,test brand @1612366146827,False,TEST BRANDCODE @1612366146827
...,...,...,...,...,...,...,...,...
1162,5f77274dbe37ce6b592e90c0,511111116752,Baking,BAKING,5f77274dbe37ce6b592e90bf,test brand @1601644365844,NaN,NaN
1163,5dc1fca91dda2c0ad7da64ae,511111706328,Breakfast & Cereal,NaN,53e10d6368abd3c7065097cc,Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL
1164,5f494c6e04db711dd8fe87e7,511111416173,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217
1165,5a021611e4b00efe02b02a57,511111400608,Grocery,NaN,5332f5f6e4b03c9a25efd0b4,LIPTON TEA Leaves,False,LIPTON TEA Leaves


For Receipt data, it contains both header level data and item level data. To simplify it, I always like to convert it to item level and remove redundant header level data.

In [625]:
# import json file to dataframe
receipts = []

with gzip.open('data/receipts.json.gz','rb') as r:
    for line in r:
        d = json.loads(line)
        receipts.append(d)
        
receipts = pd.DataFrame(receipts)

In [626]:
# convert data to item level
receipts = receipts.explode('rewardsReceiptItemList').reset_index(drop=True)

In [627]:
# fix data and simplify data
receipts['_id'] = [r['$oid'] for r in receipts['_id']]
receipts['is_modified'] = receipts['createDate']!=receipts['modifyDate']
receipts['dateScanned'] = [datetime.fromtimestamp(r['$date']/1000).strftime("%m-%d-%Y %I:%M:%S") for r in receipts['dateScanned']]
receipts['item_barcode'] = [i['barcode'] if 'barcode' in i else np.nan for i in receipts['rewardsReceiptItemList'].fillna("")]
receipts['item_brandcode'] = [i['brandCode'] if 'brandCode' in i else np.nan for i in receipts['rewardsReceiptItemList'].fillna("")]
receipts['item_description'] = [i['description'] if 'description' in i else np.nan for i in receipts['rewardsReceiptItemList'].fillna("")]
receipts['competitive_product'] = [i['competitiveProduct'] if 'competitiveProduct' in i else np.nan for i in receipts['rewardsReceiptItemList'].fillna("")]
receipts['item_quantity_purchased'] = [i['quantityPurchased'] if 'quantityPurchased' in i else np.nan for i in receipts['rewardsReceiptItemList'].fillna("")]
receipts['item_price'] = [i['itemPrice'] if 'itemPrice' in i else np.nan for i in receipts['rewardsReceiptItemList'].fillna("")]
receipts['item_final_price'] = [i['finalPrice'] if 'finalPrice' in i else np.nan for i in receipts['rewardsReceiptItemList'].fillna("")]
receipts['item_points_earned'] = [i['pointsEarned'] if 'pointsEarned' in i else np.nan for i in receipts['rewardsReceiptItemList'].fillna("")]
receipts = receipts.rename({"bonusPointsEarned":"bonus_points_earned","dateScanned":"date_scanned","pointsEarned":"points_earned","rewardsReceiptStatus":"rewards_receipt_status",}, axis=1)

In [628]:
# drop redundant fields
receipts = receipts.drop(['createDate','finishedDate','modifyDate','pointsAwardedDate','purchaseDate','purchasedItemCount','rewardsReceiptItemList','totalSpent'], axis = 1)

The receipt data now looks like tihs

In [629]:
receipts

,_id,bonus_points_earned,bonusPointsEarnedReason,date_scanned,points_earned,rewards_receipt_status,userId,is_modified,item_barcode,item_brandcode,item_description,competitive_product,item_quantity_purchased,item_price,item_final_price,item_points_earned
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",01-03-2021 07:25:31,500.0,FINISHED,5ff1e1eacfcf6c399c274ae6,True,4011,NaN,ITEM NOT FOUND,NaN,5.0,26.00,26.00,NaN
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",01-03-2021 07:24:43,150.0,FINISHED,5ff1e194b6a9d73a3a9f1052,True,4011,NaN,ITEM NOT FOUND,NaN,1.0,1,1,NaN
2,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",01-03-2021 07:24:43,150.0,FINISHED,5ff1e194b6a9d73a3a9f1052,True,028400642255,NaN,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,NaN,1.0,10.00,10.00,NaN
3,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,01-03-2021 07:25:37,5,REJECTED,5ff1e1f1cfcf6c399c274b0b,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,01-03-2021 07:25:34,5.0,FINISHED,5ff1e1eacfcf6c399c274ae6,True,4011,NaN,ITEM NOT FOUND,NaN,4.0,28.00,28.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,603d0b710a720fde1000042a,NaN,NaN,03-01-2021 07:42:41,NaN,SUBMITTED,5fc961c3b8cfca11a077dd33,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7377,603cf5290a720fde10000413,NaN,NaN,03-01-2021 06:07:37,NaN,SUBMITTED,5fc961c3b8cfca11a077dd33,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7378,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,03-01-2021 05:07:28,25.0,REJECTED,5fc961c3b8cfca11a077dd33,True,B076FJ92M4,NaN,mueller austria hypergrind precision electric ...,NaN,1.0,22.97,22.97,NaN
7379,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,03-01-2021 05:07:28,25.0,REJECTED,5fc961c3b8cfca11a077dd33,True,B07BRRLSVC,NaN,thindust summer face mask - sun protection nec...,NaN,1.0,11.99,11.99,NaN


My simplified, structured, relational data model is as below. I use barcode to join receipt data and brand data, and user uuid to join receipt and user data. 

![data_model_diagram](img/data_model_diagram.png)

# Second: Write a query that directly answers a predetermined question from a business stakeholder

At this stage, I"ll answer one of the questions that I think I have enough data so the result wouldn't mislead stakeholders.

I choose: **When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?**

In [630]:
ps.sqldf("""SELECT CASE 
                    WHEN rewards_receipt_status = 'FINISHED' 
                    THEN 'Accepted' 
                    ELSE 'Rejected' 
                    END as rewards_status, 
                   SUM(item_final_price)/COUNT(distinct _id) as [average spend] 
            FROM receipts 
            WHERE rewards_receipt_status IN ('FINISHED','REJECTED') 
            GROUP BY rewards_receipt_status""")

,rewards_status,average spend
0,Accepted,80.893842
1,Rejected,23.804930


# Third: Evaluate Data Quality Issues in the Data Provided

To join data together, joinable key is important. Brands data need to have all the barcode to be able to join with receipts data. Let see how much distinct barcode data we have in both table.

In [631]:
ps.sqldf("""SELECT COUNT(DISTINCT item_barcode) as cnt_receipts_barcode, 
            (SELECT COUNT(DISTINCT barcode) FROM brands) as cnt_brand_barcode
            FROM receipts""")

,cnt_receipts_barcode,cnt_brand_barcode
0,568,1160


We might be in good shape but let's see how much of them match.

In [632]:
ps.sqldf("""SELECT COUNT(DISTINCT item_barcode) as cnt_barcode_in_both_table
            FROM receipts r
            INNER JOIN brands b
            ON r.item_barcode = b.barcode""") 

,cnt_barcode_in_both_table
0,16


Only 16 of them match barcode in brands table.

In [633]:
ps.sqldf("""SELECT COUNT(distinct item_barcode) as cnt_barcode_missing_from_brands_table
            FROM receipts r
            LEFT JOIN brands b
            ON r.item_barcode = b.barcode
            WHERE b.barcode IS NULL""")

,cnt_barcode_missing_from_brands_table
0,552


We would need to add these 552 barcodes into brands table.

# Fourth: Communicate with Stakeholders

Hi product manager,

After reviewing all the data we have here, I have a few questions would like your advice.
I wonder whether brand information such as category and cpg is import at this stage of analysis, as I notice that we might only have a subset of brands data. I tried to join receipt data with brand, but only 16 barcodes in receipts data could be found in brands data. If it's important, I wonder who or which department could I reach out to pull all the brands data we have in datawarehouse?

Also, if it's possible, I hope we could put together **receipt issued merchant data**, as sometimes reward benefit more on retail store like Walmart than specific brand. With the merchant data, we might find something we never think about. For this type of issue, going forward I will create a data validation script to check whether we are missing brands/merchant records in cross-reference data and send them to data collection team. When we scaling up, we would see more of this issue happen but I"ll do my best to report it in advance.

Sincerely,
Yu-Wei Tang